## Basic Tweet Cleaning for NLP

In [1]:
import re

import pandas as pd
import numpy as np
import spacy 

In [10]:
# tweets from 01-01-2017 to 04-11-2019
raw = pd.read_json('../data/tweets/realdonaldtrump.json')

raw.rename({'full_text':'raw_text'}, axis=1, inplace=True)
raw.index = pd.to_datetime(raw.created_at)

In [14]:
raw.head()

,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,raw_text,id,id_str,in_reply_to_screen_name,...,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,source,truncated,user
created_at,,,,,,,,,,,,,,,,,,,,,
2018-10-31 19:24:22,2018-10-31 19:24:22,"[0, 264]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,57519,False,Yesterday in Pittsburgh I was really impressed...,1057714919766925314,1057714919766925312,None,...,NaN,NaN,NaN,NaN,NaN,13036,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':..."
2018-01-25 00:27:00,2018-01-25 00:27:00,"[0, 209]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,138214,False,"Will soon be heading to Davos, Switzerland, to...",956322473955545089,956322473955545088,None,...,NaN,NaN,NaN,NaN,NaN,24215,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':..."
2018-09-22 01:05:00,2018-09-22 01:05:00,"[0, 56]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,55764,False,Thank you Missouri - I love you!\nhttps://t.co...,1043305127250194432,1043305127250194432,None,...,0.0,NaN,NaN,NaN,NaN,13068,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':..."
2018-01-04 11:11:45,2018-01-04 11:11:45,"[0, 248]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,139844,False,"As Americans, you need identification, sometim...",948874586006925313,948874586006925312,None,...,NaN,NaN,NaN,NaN,NaN,34630,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':..."
2017-06-16 22:32:20,2017-06-16 22:32:20,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 875843499811319811, 'id_str'...",39011,False,"""National Security Presidential Memorandum on ...",875843513761583104,875843513761583104,None,...,0.0,NaN,NaN,NaN,NaN,8070,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':..."


In [12]:
raw.describe()

,contributors,coordinates,favorite_count,geo,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,possibly_sensitive,quoted_status_id,quoted_status_id_str,retweet_count
count,0.0,0.0,9053.000000,0.0,9.053000e+03,9.053000e+03,6.060000e+02,6.060000e+02,6.150000e+02,6.150000e+02,2535.000000,5.710000e+02,5.710000e+02,9053.000000
mean,NaN,NaN,83317.458632,NaN,1.039574e+18,1.039574e+18,1.117570e+18,1.117570e+18,1.335727e+15,1.335727e+15,0.000394,1.100787e+18,1.100787e+18,18896.678781
std,NaN,NaN,39521.916142,NaN,1.085005e+17,1.085005e+17,8.134106e+16,8.134106e+16,3.312496e+16,3.312496e+16,0.019861,1.046113e+17,1.046113e+17,10577.678810
min,NaN,NaN,2424.000000,NaN,8.154223e+17,8.154223e+17,8.225018e+17,8.225018e+17,7.592510e+05,7.592510e+05,0.000000,3.009181e+16,3.009181e+16,586.000000
25%,NaN,NaN,58546.000000,NaN,9.491265e+17,9.491265e+17,1.107258e+18,1.107258e+18,2.507388e+07,2.507388e+07,0.000000,1.041827e+18,1.041827e+18,12789.000000
50%,NaN,NaN,76325.000000,NaN,1.052522e+18,1.052522e+18,1.149282e+18,1.149282e+18,2.507388e+07,2.507388e+07,0.000000,1.146947e+18,1.146947e+18,17002.000000
75%,NaN,NaN,99121.000000,NaN,1.138404e+18,1.138404e+18,1.169357e+18,1.169357e+18,2.507388e+07,2.507388e+07,0.000000,1.175987e+18,1.175987e+18,22470.000000
max,NaN,NaN,869172.000000,NaN,1.191410e+18,1.191410e+18,1.191360e+18,1.191360e+18,8.214724e+17,8.214724e+17,1.000000,1.191194e+18,1.191194e+18,313949.000000


In [13]:
raw = raw.drop(['contributors', 'coordinates','geo'], axis=1)

In [21]:
for i in raw.raw_text[:10]:
    print(i)

Yesterday in Pittsburgh I was really impressed with Congressman Keith Rothfus (far more so than any other local political figure). His sincere level of compassion, grief and sorrow for the events that took place was, in its own way, very inspiring. Vote for Keith!
Will soon be heading to Davos, Switzerland, to tell the world how great America is and is doing. Our economy is now booming and with all I am doing, will only get better...Our country is finally WINNING again!
Thank you Missouri - I love you!
https://t.co/rx3mtd7yOy
As Americans, you need identification, sometimes in a very strong and accurate form, for almost everything you do.....except when it comes to the most important thing, VOTING for the people that run your country. Push hard for Voter Identification!
"National Security Presidential Memorandum on Strengthening the Policy of the United States Toward Cuba"
Memorandum➡️https://t.co/DlQTEMRI6T https://t.co/o5MhFdKCJL
“Serial killers get more Due Process than the Democrat

In [22]:
raw.entities[4]

{'hashtags': [],
 'symbols': [],
 'user_mentions': [],
 'urls': [{'url': 'https://t.co/DlQTEMRI6T',
   'expanded_url': 'http://45.wh.gov/of9ceK',
   'display_url': '45.wh.gov/of9ceK',
   'indices': [117, 140]}],
 'media': [{'id': 875843499811319811,
   'id_str': '875843499811319811',
   'indices': [141, 164],
   'media_url': 'http://pbs.twimg.com/media/DCefBc0WsAMjh-H.jpg',
   'media_url_https': 'https://pbs.twimg.com/media/DCefBc0WsAMjh-H.jpg',
   'url': 'https://t.co/o5MhFdKCJL',
   'display_url': 'pic.twitter.com/o5MhFdKCJL',
   'expanded_url': 'https://twitter.com/realDonaldTrump/status/875843513761583104/photo/1',
   'type': 'photo',
   'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
    'small': {'w': 680, 'h': 539, 'resize': 'fit'},
    'medium': {'w': 1008, 'h': 799, 'resize': 'fit'},
    'large': {'w': 1008, 'h': 799, 'resize': 'fit'}},
   'features': {'orig': {'faces': [{'x': 462, 'y': 0, 'h': 33, 'w': 33},
      {'x': 80, 'y': 15, 'h': 36, 'w': 36},
      {'x': 50

In [23]:
# create three new columns: names mentioned (@...), hashtags used (#...), links ('http...')

def find_hashtags(tweet):
    hash_regex = re.compile(r'#\w*')
    return hash_regex.findall(tweet)

def find_mentions(tweet):
    mention_regex = re.compile(r'@\w*')
    return mention_regex.findall(tweet)

def find_links(tweet):
    link_regex = re.compile(r'http\S*')
    return link_regex.findall(tweet)
    


In [24]:
print(raw.raw_text[4]) 

"National Security Presidential Memorandum on Strengthening the Policy of the United States Toward Cuba"
Memorandum➡️https://t.co/DlQTEMRI6T https://t.co/o5MhFdKCJL


In [25]:
print(find_hashtags(raw.raw_text[4]))
print(find_mentions(raw.raw_text[4]))
print(find_links(raw.raw_text[4]))

[]
[]
['https://t.co/DlQTEMRI6T', 'https://t.co/o5MhFdKCJL']


In [26]:
raw['hashtags'] = raw.raw_text.apply(find_hashtags)
raw['mentions'] = raw.raw_text.apply(find_mentions)
raw['links'] = raw.raw_text.apply(find_links)
raw.head()

,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,raw_text,id,id_str,in_reply_to_screen_name,...,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,source,truncated,user,hashtags,mentions,links
created_at,,,,,,,,,,,,,,,,,,,,,
2018-10-31 19:24:22,2018-10-31 19:24:22,"[0, 264]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,57519,False,Yesterday in Pittsburgh I was really impressed...,1057714919766925314,1057714919766925312,None,...,NaN,NaN,13036,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':...",[],[],[]
2018-01-25 00:27:00,2018-01-25 00:27:00,"[0, 209]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,138214,False,"Will soon be heading to Davos, Switzerland, to...",956322473955545089,956322473955545088,None,...,NaN,NaN,24215,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':...",[],[],[]
2018-09-22 01:05:00,2018-09-22 01:05:00,"[0, 56]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,55764,False,Thank you Missouri - I love you!\nhttps://t.co...,1043305127250194432,1043305127250194432,None,...,NaN,NaN,13068,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':...",[],[],[https://t.co/rx3mtd7yOy]
2018-01-04 11:11:45,2018-01-04 11:11:45,"[0, 248]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,139844,False,"As Americans, you need identification, sometim...",948874586006925313,948874586006925312,None,...,NaN,NaN,34630,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':...",[],[],[]
2017-06-16 22:32:20,2017-06-16 22:32:20,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 875843499811319811, 'id_str'...",39011,False,"""National Security Presidential Memorandum on ...",875843513761583104,875843513761583104,None,...,NaN,NaN,8070,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':...",[],[],"[https://t.co/DlQTEMRI6T, https://t.co/o5MhFdK..."


In [27]:
# now let's clean the actual tweet so it's just the message

# removes RT header ('RT @IvankaTrump: '), links, and hashtags
# leave instances of '@HillaryClinton?' or 'MikePence!' since these
# are directed *at* people. Will tag as entities later through nlp
    
def clean_tweet(df):
    word_list = ['RT'] + df.hashtags + [mention+':' for mention in df.mentions] + df.links
    
    raw_tweet = df.raw_text
    try:
        for stop in word_list:
            raw_tweet = raw_tweet.replace(stop, '')
        return raw_tweet
    except:
        return np.nan
        
    

In [28]:
raw['clean_text'] = raw.apply(clean_tweet, axis=1)

In [29]:
raw.head()

,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,raw_text,id,id_str,in_reply_to_screen_name,...,quoted_status_permalink,retweet_count,retweeted,source,truncated,user,hashtags,mentions,links,clean_text
created_at,,,,,,,,,,,,,,,,,,,,,
2018-10-31 19:24:22,2018-10-31 19:24:22,"[0, 264]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,57519,False,Yesterday in Pittsburgh I was really impressed...,1057714919766925314,1057714919766925312,None,...,NaN,13036,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':...",[],[],[],Yesterday in Pittsburgh I was really impressed...
2018-01-25 00:27:00,2018-01-25 00:27:00,"[0, 209]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,138214,False,"Will soon be heading to Davos, Switzerland, to...",956322473955545089,956322473955545088,None,...,NaN,24215,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':...",[],[],[],"Will soon be heading to Davos, Switzerland, to..."
2018-09-22 01:05:00,2018-09-22 01:05:00,"[0, 56]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,55764,False,Thank you Missouri - I love you!\nhttps://t.co...,1043305127250194432,1043305127250194432,None,...,NaN,13068,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':...",[],[],[https://t.co/rx3mtd7yOy],Thank you Missouri - I love you!\n
2018-01-04 11:11:45,2018-01-04 11:11:45,"[0, 248]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,139844,False,"As Americans, you need identification, sometim...",948874586006925313,948874586006925312,None,...,NaN,34630,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':...",[],[],[],"As Americans, you need identification, sometim..."
2017-06-16 22:32:20,2017-06-16 22:32:20,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 875843499811319811, 'id_str'...",39011,False,"""National Security Presidential Memorandum on ...",875843513761583104,875843513761583104,None,...,NaN,8070,False,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 25073877, 'id_str': '25073877', 'name':...",[],[],"[https://t.co/DlQTEMRI6T, https://t.co/o5MhFdK...","""National Security Presidential Memorandum on ..."


In [30]:
print(raw.raw_text[4], '\n')
print(raw.clean_text[4])

"National Security Presidential Memorandum on Strengthening the Policy of the United States Toward Cuba"
Memorandum➡️https://t.co/DlQTEMRI6T https://t.co/o5MhFdKCJL 

"National Security Presidential Memorandum on Strengthening the Policy of the United States Toward Cuba"
Memorandum➡️ 


In [40]:
nlp = spacy.load('en_core_web_sm')

In [41]:
# saves each tweet as a spacy 'doc'
raw['doc'] = raw.clean_text.apply(nlp)

In [42]:
# create word vectors for each cleaned tweet
raw['vector'] = raw.doc.apply(lambda x: x.vector)

def find_ents(doc):
    return [ent.text for ent in doc.ents]
raw['entities'] = raw.doc.apply(find_ents)

In [43]:
raw[['doc','vector','entities']].head()

,doc,vector,entities
created_at,,,
2018-10-31 19:24:22,"(Yesterday, in, Pittsburgh, I, was, really, im...","[-0.78020597, -0.15337843, -1.2758473, -1.4249...","[Yesterday, Pittsburgh, Keith Rothfus, Keith]"
2018-01-25 00:27:00,"(Will, soon, be, heading, to, Davos, ,, Switze...","[0.6131167, -0.4810254, -0.48694012, -1.765091...","[Davos, Switzerland, America]"
2018-09-22 01:05:00,"(Thank, you, Missouri, -, I, love, you, !, \n)","[-1.1658549, -0.28248873, -0.5705499, -0.69825...",[Missouri]
2018-01-04 11:11:45,"(As, Americans, ,, you, need, identification, ...","[-0.22083983, -0.23803626, -0.63418156, -1.644...","[Americans, VOTING]"
2017-06-16 22:32:20,"("", National, Security, Presidential, Memorand...","[-1.4124, 1.422501, -0.8739315, -1.612236, -0....",[]


In [45]:
len(str(raw.doc[0]).split())

44

In [46]:
len(raw.doc[0])


52

In [47]:
for token in raw.doc[0]:
    print(token.pos_)

NOUN
ADP
PROPN
PRON
AUX
ADV
ADJ
ADP
PROPN
PROPN
PROPN
PUNCT
ADV
ADV
ADV
SCONJ
DET
ADJ
ADJ
ADJ
NOUN
PUNCT
PUNCT
PRON
ADJ
NOUN
ADP
NOUN
PUNCT
NOUN
CCONJ
NOUN
ADP
DET
NOUN
PRON
VERB
NOUN
AUX
PUNCT
ADP
PRON
ADJ
NOUN
PUNCT
ADV
ADJ
PUNCT
NOUN
ADP
PROPN
PUNCT


In [58]:
for i in raw.doc[0]:
    print(i, i.pos_.rjust(20-len(i)))

Yesterday        NOUN
in                ADP
Pittsburgh      PROPN
I                PRON
was               AUX
really            ADV
impressed         ADJ
with              ADP
Congressman     PROPN
Keith           PROPN
Rothfus         PROPN
(               PUNCT
far               ADV
more              ADV
so                ADV
than            SCONJ
any               DET
other             ADJ
local             ADJ
political         ADJ
figure           NOUN
)               PUNCT
.               PUNCT
His              PRON
sincere           ADJ
level            NOUN
of                ADP
compassion       NOUN
,               PUNCT
grief            NOUN
and             CCONJ
sorrow           NOUN
for               ADP
the               DET
events           NOUN
that             PRON
took             VERB
place            NOUN
was               AUX
,               PUNCT
in                ADP
its              PRON
own               ADJ
way              NOUN
,               PUNCT
very      

In [48]:
spacy.explain('PART')

'particle'

In [49]:
print(pd.DataFrame.to_pickle.__doc__)


        Pickle (serialize) object to file.

        Parameters
        ----------
        path : str
            File path where the pickled object will be stored.
        compression : {'infer', 'gzip', 'bz2', 'zip', 'xz', None},         default 'infer'
            A string representing the compression to use in the output file. By
            default, infers from the file extension in specified path.

            .. versionadded:: 0.20.0
        protocol : int
            Int which indicates which protocol should be used by the pickler,
            default HIGHEST_PROTOCOL (see [1]_ paragraph 12.1.2). The possible
            values for this parameter depend on the version of Python. For
            Python 2.x, possible values are 0, 1, 2. For Python>=3.0, 3 is a
            valid value. For Python >= 3.4, 4 is a valid value. A negative
            value for the protocol parameter is equivalent to setting its value
            to HIGHEST_PROTOCOL.

            .. [1] https://docs.py

In [50]:
# pickle the dataframe so Docs are saved for later use
raw.to_pickle('../data/tweets/clean_tweets.pkl')